In [14]:
!pip install random-word

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 11.2 MB/s eta 0:00:00


In [265]:
import random
def get_randomize_string(string, num_chars):
    for _ in range(num_chars):
        index = random.randint(0, len(string) - 1)
        string = string.replace(string[index], "")
    return string

def generate_combinations(lists, current_combination, index, result, list_orig, triger, label):
    if index == len(lists):
        count = 0
        target = 0
        comb_str = ' '.join(current_combination.copy())
        for item in list_orig:
            if item in comb_str:
                count += 1
                if count > triger:
                    target = label
                    break
        current_combination_buf = ""
        for item in current_combination:
            chance = random.randint(0, 3) - 1
            if chance == -1:
                chance = 0
            string_changed = get_randomize_string(item, chance)
            current_combination_buf += " " + string_changed
        result.append((current_combination_buf, target))
        return

    for word in lists[index]:
        current_combination.append(word)
        generate_combinations(lists, current_combination, index + 1, result, list_orig, triger, label)
        current_combination.pop()

# Функция для перебора комбинаций слов из разных списков
def all_combinations(lists, list_orig, triger, label):
    result = []
    generate_combinations(lists, [], 0, result, list_orig, triger, label)
    return result

In [266]:
# чтение данных
import pandas as pd

df = pd.read_csv("output.csv")

import csv
lists_buf = []
with open("output.csv", "r") as f:
    reader = csv.reader(f, delimiter=",")
    for line in reader:
        if "" in line:
            index = line.index('')
            lists_buf.append(line[:index])
        else:
            lists_buf.append(line)

lists_person = []
with open("output.csv", "r") as f:
    reader = csv.reader(f, delimiter=",")
    for line in reader:
        lists_person.append(line)

list_orig_person = []
for item in lists_buf:
    list_orig_person += item

In [267]:
# Создание шаблонных примеров заголовков
list_orig_fin = ["pay", "check", "bill", "credit", "dept","series", "seria", "obligation", "sum"]
list_orig_bio = ["finger", "eye", "snapshot", "photo"]
lists_fin = [
    ["pay", "payment", "check", "bill", "sum", "", ""],
    ["credit", "dept", "obligation", "", "", "", "", ""],
    ["seria", "series", "", "", "", "", "", ""],
]
lists_bio = [
    ["finger", "fingerprint", "", "", "", ""],
    ["eye", "eyeprint", "", "", "", ""],
    ["photo", "snapshot", "", "", "", ""],
]

all_lists = [lists_person, lists_fin, lists_bio]
all_origs = [list_orig_person, list_orig_fin, list_orig_bio]

In [268]:
# Заполнение пропусков случайными словами
from random_word import RandomWords
r = RandomWords()


for i in range(len(all_lists)):
    for j in range(len(all_lists[i])):
        for k in range(len(all_lists[i][j])):
            if all_lists[i][j][k] == '':
                all_lists[i][j][k] = r.get_random_word()

In [269]:
# Построение датасета на основе комбинаций из шаблона
data = []
for i in range(len(all_lists)):
    data_ = []
    if i == 0:
        len_ = 5
    else:
        len_ = len(all_lists[i])
    for j in range(1, len_):
        handler = all_lists[i][:j]
        for l in all_lists[i][j:]:
            data_.append(all_combinations(handler + [l], all_origs[i], triger=2-i, label=i+1))

    data_buf = []
    for item in all_origs[i]:
        label = i + 1 if i else 0
        data_buf.append((item, i+1))
    data += [data_buf] + data_

In [270]:
# Преобразование в удобный для записи в файл вид
data_ = []
for item in data:
    data_ += item

In [273]:
# Запись в csv файл датасета
with open("data.csv", "w") as f:
    writer = csv.writer(f, delimiter=';')
    for row in data_:
        writer.writerow(row[0] + "," + str(row[1]))